In [1]:
import cv2
import numpy as np
import os

# Create an output directory if it doesn't exist
output_dir = "heightimage"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create a text file to write results (e.g., leaf counts or plant height)
result_file = open("plant_results.txt", "w")

# Directory with input images
input_dir = "INPUT2"  # Update this with your input directory path

def calculate_plant_height(image_path, reference_height_cm=None, reference_pixels=None):
    """Calculates plant height in pixels or real-world units (cm)."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # --- Segmentation (Color Masking - Adapt as needed) ---
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 40, 40])  # Adjust these values for better plant isolation
    upper_green = np.array([85, 255, 255])  # Adjust based on your plant's green color range
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # --- Apply Morphological Transformations to clean up the mask ---
    kernel = np.ones((5, 5), np.uint8)  # Adjust kernel size for more aggressive cleaning
    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)  # Removes small noise
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel)  # Fills gaps

    thresh = mask_cleaned

    # --- Contour Detection ---
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_area = 0
    stem_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            stem_contour = contour

    if stem_contour is None:
        print(f"Error: No stem contour found in {image_path}.")
        return None

    # --- Height Calculation (Convex Hull - More Accurate) ---
    hull = cv2.convexHull(stem_contour)
    top_point = tuple(hull[hull[:, :, 1].argmin()][0])
    bottom_point = tuple(hull[hull[:, :, 1].argmax()][0])
    stem_height_pixels = abs(bottom_point[1] - top_point[1])

    # --- Calibration (if reference object is provided) ---
    if reference_height_cm is not None and reference_pixels is not None:
        pixel_to_cm = reference_height_cm / reference_pixels
        stem_height_cm = stem_height_pixels * pixel_to_cm
        return stem_height_cm
    else:
        return stem_height_pixels

# Iterate over all files in the input directory
for filename in os.listdir(input_dir):
    # Check if the file is an image (you can add more file types if needed)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load image
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)

        # Convert to HSV (Hue, Saturation, Value) color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define the range of green color in HSV space for better plant isolation
        lower_green = np.array([35, 40, 40])  # Adjust these values for more isolation
        upper_green = np.array([85, 255, 255])

        # Create a mask that isolates the green regions (leaves and stem)
        mask = cv2.inRange(hsv, lower_green, upper_green)

        # Apply morphological operations to clean up the mask
        kernel = np.ones((5, 5), np.uint8)
        mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)  # Removes noise
        mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel)  # Fills gaps

        # Find contours on the cleaned mask
        contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours based on area (ignore small noise)
        min_plant_area = 100  # Minimum size to be considered a plant part
        plant_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_plant_area]

        # Draw contours on the original image for visualization
        output = image.copy()
        cv2.drawContours(output, plant_contours, -1, (0, 255, 0), 2)

        # Count plant parts (leaf + stem)
        plant_part_count = len(plant_contours)

        # Write the plant part count to the result text file
        result_file.write(f"{filename}: {plant_part_count} plant parts\n")

        # Save the output image with contours
        output_image_path = os.path.join(output_dir, f"contours_{filename}")
        cv2.imwrite(output_image_path, output)

        # Optionally, calculate plant height if needed
        height = calculate_plant_height(image_path)  # Optionally, provide reference data
        if height is not None:
            result_file.write(f"{filename}: Plant height = {height:.2f} cm\n")

# Close the result file after writing the results
result_file.close()

# Destroy all OpenCV windows (if any are open)
cv2.destroyAllWindows()

Error: No stem contour found in INPUT2\DAY01-05-30.jpg.
Error: No stem contour found in INPUT2\DAY01-13-30.jpg.
Error: No stem contour found in INPUT2\DAY01-21-30.jpg.
Error: No stem contour found in INPUT2\DAY02-05-30.jpg.
Error: No stem contour found in INPUT2\DAY02-13-30.jpg.
Error: No stem contour found in INPUT2\DAY02-21-30.jpg.
Error: No stem contour found in INPUT2\DAY03-05-30.jpg.
Error: No stem contour found in INPUT2\DAY03-13-30.jpg.
Error: No stem contour found in INPUT2\DAY03-21-30.jpg.
Error: No stem contour found in INPUT2\DAY04-05-30.jpg.
Error: No stem contour found in INPUT2\DAY04-13-30.jpg.
Error: No stem contour found in INPUT2\DAY04-21-30.jpg.
